In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve
import math
np.random.seed(20170306)
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
# EDA
X_train = pd.read_csv("midterm_train.csv")
y_train = X_train.pop('y')
X_test = pd.read_csv("midterm_test.csv")

In [3]:
X_train.describe()

C:\Users\leeda\Anaconda3\lib\site-packages\numpy\lib\function_base.py:3834: RuntimeWarning: Invalid value encountered in percentile
  RuntimeWarning)


,x0,x1,x2,x3,x4,x5,x6,x7,x8,x10,...,x38,x39,x40,x41,x42,x45,x46,x47,x48,x49
count,159964.000000,159973.000000,159973.000000,159963.000000,159969.000000,159971.000000,159973.000000,159974.000000,159965.000000,159962.000000,...,159960.000000,159952.000000,159969.000000,159980.000000,159965.000000,159972.000000,159974.000000,159970.000000,159974.000000,159974.000000
mean,1.540669,2.020926,0.007805,2.471131,0.004914,0.100313,0.020943,0.000992,0.004064,-0.004841,...,-0.010644,0.019918,-0.014515,-0.004971,0.006440,0.882868,0.070251,0.072817,-0.004404,-0.004627
std,16.401459,7.462316,6.806854,9.624773,6.275538,2.173207,7.753960,2.414990,9.165963,2.845066,...,8.982346,6.698498,4.973109,5.428919,6.743532,23.987879,17.732750,18.248654,4.468600,3.610345
min,-68.981217,-31.258618,-30.506154,-36.350492,-27.575136,-13.677304,-32.781282,-11.615171,-38.961689,-11.978764,...,-39.316884,-27.076148,-23.265768,-24.602106,-32.319316,-121.292948,-72.694807,-74.809917,-20.696774,-16.991354
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
max,70.882303,39.193550,28.699898,47.313496,28.571894,10.006106,34.460095,10.794778,39.216249,13.644499,...,42.025025,29.862557,23.132292,22.938242,28.980718,136.368013,77.429817,79.682696,19.217507,15.117022


In [4]:
def describe_categorical(X):
    """
    Just like .describe(), but returns the results for
    categorical variables only.
    """
    from IPython.display import display, HTML
    display(HTML(X[X.columns[X.dtypes == "object"]].describe().to_html()))
describe_categorical(X_train)

,x9,x16,x19,x43,x44
count,159966,159964,159971,159973,159961
unique,12,3,12,5,129468
top,0.01%,asia,Jun,wednesday,$48.84
freq,38827,138509,44659,101016,6


In [5]:
# Strip unwanted % and $
# http://stackoverflow.com/questions/13682044/pandas-dataframe-remove-unwanted-parts-from-strings-in-a-column
X_train['x9'] = pd.to_numeric(X_train['x9'].map(lambda x: str(x)[:-1]), errors='coerce')/100
X_train['x44'] = pd.to_numeric(X_train['x44'].map(lambda x: str(x)[1:]), errors='coerce')

In [6]:
# Fill missing values
cols = list(X_train.describe())
for x in cols:
    avg = X_train[x].mean()
    X_train[x] = X_train[x].fillna(value=avg)

C:\Users\leeda\Anaconda3\lib\site-packages\numpy\lib\function_base.py:3834: RuntimeWarning: Invalid value encountered in percentile
  RuntimeWarning)


In [7]:
X_train.describe()

,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,...,x39,x40,x41,x42,x44,x45,x46,x47,x48,x49
count,160000.000000,160000.000000,160000.000000,160000.000000,160000.000000,160000.000000,160000.000000,160000.000000,160000.000000,1.600000e+05,...,160000.000000,160000.000000,160000.000000,160000.000000,160000.000000,160000.000000,160000.000000,160000.000000,160000.000000,160000.000000
mean,1.540669,2.020926,0.007805,2.471131,0.004914,0.100313,0.020943,0.000992,0.004064,9.126939e-08,...,0.019918,-0.014515,-0.004971,0.006440,-0.587294,0.882868,0.070251,0.072817,-0.004404,-0.004627
std,16.399614,7.461686,6.806280,9.623660,6.274930,2.173010,7.753306,2.414794,9.164960,1.039774e-04,...,6.697493,4.972627,5.428579,6.742794,999.159564,23.985780,17.731309,18.246943,4.468236,3.610051
min,-68.981217,-31.258618,-30.506154,-36.350492,-27.575136,-13.677304,-32.781282,-11.615171,-38.961689,-5.000000e-04,...,-27.076148,-23.265768,-24.602106,-32.319316,-4343.800000,-121.292948,-72.694807,-74.809917,-20.696774,-16.991354
25%,-9.446061,-2.888700,-4.608261,-4.028796,-4.224946,-1.375042,-5.196455,-1.639108,-6.157372,-1.000000e-04,...,-4.505689,-3.380758,-3.667926,-4.522596,-672.095000,-14.593806,-12.238163,-12.592695,-3.019633,-2.438531
50%,1.837036,2.199865,0.008477,2.581708,-0.004281,0.058530,-0.016175,0.000992,0.241001,0.000000e+00,...,0.025682,-0.000038,-0.016829,0.009630,0.875000,0.884875,-0.444714,-0.457144,-0.001562,-0.002179
75%,12.784307,7.001267,4.622527,9.014487,4.256174,1.550318,5.267178,1.627476,6.256527,1.000000e-04,...,4.548195,3.344241,3.649401,4.561391,671.157500,16.554647,11.985884,12.334623,3.002881,2.424336
max,70.882303,39.193550,28.699898,47.313496,28.571894,10.006106,34.460095,10.794778,39.216249,5.000000e-04,...,29.862557,23.132292,22.938242,28.980718,4929.460000,136.368013,77.429817,79.682696,19.217507,15.117022


In [8]:
describe_categorical(X_train)

,x16,x19,x43
count,159964,159971,159973
unique,3,12,5
top,asia,Jun,wednesday
freq,138509,44659,101016


In [9]:
categorical_variables = X_train.columns[X_train.dtypes == "object"].tolist()
for variable in categorical_variables:
    # Fill missing data with the word "Missing"
    X_train[variable].fillna("Missing", inplace=True)
    # Create array of dummies
    dummies = pd.get_dummies(X_train[variable], prefix=variable)
    # Update X to include dummies and drop the main variable
    X_train = pd.concat([X_train, dummies], axis=1)
    X_train.drop([variable], axis=1, inplace=True)

In [10]:
### Clean Test Set
# Strip unwanted % and $
# http://stackoverflow.com/questions/13682044/pandas-dataframe-remove-unwanted-parts-from-strings-in-a-column
X_test['x9'] = pd.to_numeric(X_test['x9'].map(lambda x: str(x)[:-1]), errors='coerce')/100
X_test['x44'] = pd.to_numeric(X_test['x44'].map(lambda x: str(x)[1:]), errors='coerce')

# Fill missing values
cols = list(X_test.describe())
for x in cols:
    avg = X_test[x].mean()
    X_test[x] = X_test[x].fillna(value=avg)

# Dummies
categorical_variables = X_test.columns[X_test.dtypes == "object"].tolist()
for variable in categorical_variables:
    # Fill missing data with the word "Missing"
    X_test[variable].fillna("Missing", inplace=True)
    # Create array of dummies
    dummies = pd.get_dummies(X_test[variable], prefix=variable)
    # Update X to include dummies and drop the main variable
    X_test = pd.concat([X_test, dummies], axis=1)
    X_test.drop([variable], axis=1, inplace=True)

C:\Users\leeda\Anaconda3\lib\site-packages\numpy\lib\function_base.py:3834: RuntimeWarning: Invalid value encountered in percentile
  RuntimeWarning)


In [11]:
# Confirm Data is Cleaned
# Look at all the columns in the dataset
def printall(X, max_rows=10):
    from IPython.display import display, HTML
    display(HTML(X.to_html(max_rows=max_rows)))
    
printall(X_train)

,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x17,x18,x20,x21,x22,x23,x24,x25,x26,x27,x28,x29,x30,x31,x32,x33,x34,x35,x36,x37,x38,x39,x40,x41,x42,x44,x45,x46,x47,x48,x49,x16_Missing,x16_america,x16_asia,x16_euorpe,x19_Apr,x19_Aug,x19_Dev,x19_Feb,x19_January,x19_July,x19_Jun,x19_Mar,x19_May,x19_Missing,x19_Nov,x19_Oct,x19_sept.,x43_Missing,x43_friday,x43_monday,x43_thurday,x43_tuesday,x43_wednesday
0,-5.900707,0.469389,4.733068,9.223027,3.170787,0.024669,-2.055880,-2.045466,3.004170,0.0002,-4.685655,5.895884,12.235718,-17.658996,-19.322599,1.978951,0.831200,1.980735,-1.347138,0.320392,-1.177156,-2.704137,-14.103989,-5.655715,5.481989,-1.161496,-3.931582,7.643024,3.421198,3.762529,8.698367,-6.727415,-10.732802,-0.728671,0.018602,-32.680226,-0.106695,3.913680,-5.335641,-3.694592,-1.008670,-180.16,1.236134,4.462951,4.592803,-0.034478,3.680401,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,-3.530391,4.229818,-4.619433,2.154913,5.619903,-1.955667,7.713001,-1.279395,14.941269,0.0000,-3.103170,0.982095,3.753298,7.537105,5.942761,0.088104,0.022300,-5.310811,0.898115,-1.935222,-0.398599,-0.979549,-10.746822,2.507350,1.280838,3.452034,0.624896,0.426659,2.874107,18.712973,7.312360,-15.853969,-1.455398,0.236479,-3.252878,-1.289889,14.377643,6.201034,-8.490805,6.414948,5.831287,432.31,-6.570015,-5.308026,-5.462467,-0.229518,-2.332947,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1.003389,-0.012547,6.250503,3.368675,1.711321,6.192718,4.166159,-0.363055,15.175681,-0.0001,-5.424805,7.969230,-16.521783,-10.213318,9.272947,1.128275,0.725844,-10.495085,4.546223,2.019324,4.339569,-0.205385,-15.868465,-0.501901,2.002275,0.225154,-0.882617,-14.174509,-14.707616,19.006559,2.732869,-35.200541,-21.825129,0.365506,-5.381501,16.668153,-7.932493,-0.397368,-4.172583,0.371862,-7.025987,-119.17,-66.851394,-11.084558,-11.407072,6.395551,3.545332,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,-16.041769,-3.243507,-0.546649,-11.594427,5.111965,0.609785,-1.540136,-1.227567,-16.460535,-0.0000,5.624369,9.138195,2.538420,14.740934,10.200809,-2.862860,-0.358076,2.548242,-1.207699,-1.183008,-11.578898,-2.626346,15.628244,-16.420842,-6.891503,-4.772674,-0.172465,2.928158,-11.267913,-20.615756,5.138457,20.096343,11.691170,0.442023,-0.380077,29.446823,10.975024,1.148454,1.666316,-2.920478,-2.135088,-1309.54,0.442939,11.808383,12.151957,-2.923916,-1.471336,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,20.498575,10.766191,1.478032,-5.723594,-0.768839,-3.482515,-11.169389,-0.361478,2.194909,-0.0002,1.331710,13.616031,2.503163,-21.545901,-31.082999,0.964522,-0.340416,-4.412438,-0.504193,0.198800,-1.763387,3.767706,-13.312645,-6.421953,-3.401994,-2.892450,-5.740850,7.334497,0.836239,2.748982,7.637753,-15.819318,-7.478481,0.470682,-2.954060,-10.051513,4.656588,-5.186225,-0.757455,-1.769904,-0.532896,-1103.32,61.136654,48.077895,49.476758,1.043061,9.927952,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159995,-3.770242,2.560371,4.362929,14.250854,-4.870163,1.970673,8.472495,-1.783486,-7.071905,-0.0001,1.868205,12.561893,-4.208149,-25.376775,-20.569176,4.000076,0.471655,13.031095,-3.255901,-2.082249,-6.170630,-4.894947,-12.724050,10.047053,8.470432,5.552013,-0.439915,5.869206,-6.236911,-8.857104,18.884906,7.249665,9.945951,1.705074,-7.608455,-26.463533,-4.933390,-9.297190,6.931556,-0.680760,-2.728167,-14.70,20.411108,16.430708,16.908772,-1.618275,-3.172172,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
159996,14.763

In [12]:
printall(X_test)

,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x17,x18,x20,x21,x22,x23,x24,x25,x26,x27,x28,x29,x30,x31,x32,x33,x34,x35,x36,x37,x38,x39,x40,x41,x42,x44,x45,x46,x47,x48,x49,x16_Missing,x16_america,x16_asia,x16_euorpe,x19_Apr,x19_Aug,x19_Dev,x19_Feb,x19_January,x19_July,x19_Jun,x19_Mar,x19_May,x19_Missing,x19_Nov,x19_Oct,x19_sept.,x43_Missing,x43_friday,x43_monday,x43_thurday,x43_tuesday,x43_wednesday
0,13.864410,0.986217,6.835099,12.129012,1.141835,0.257978,6.119007,-0.217401,-0.913796,0.0000,-0.419751,10.172915,-6.886273,-9.130198,-4.578509,7.960524,0.111165,-9.699845,-3.387555,1.911904,0.998081,-0.857490,-7.612868,-4.018814,7.209250,3.013906,-1.446528,-8.230251,-2.442596,-1.144471,-4.253401,-5.203326,2.729897,-1.202386,-2.511575,-13.927587,-14.171462,-2.149515,-11.997662,-3.078159,4.129892,376.12,21.431275,10.409146,10.712008,4.278259,-6.113483,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,17.075344,-8.614119,9.730068,3.645163,18.618751,-1.134351,1.324885,2.750353,-21.323635,-0.0001,1.040758,-8.328905,21.474338,-82.372525,-19.870239,1.919702,0.513677,0.667307,2.426892,4.125267,-3.196495,1.820393,-30.719266,-0.332577,2.166615,-4.156138,-2.056772,1.107974,-7.752646,-26.706473,0.255282,20.459172,-19.764989,-0.103611,4.232745,-49.751647,8.288716,12.060118,0.782677,4.364007,-2.506411,1058.58,31.912299,23.853665,24.547705,1.835443,-0.698576,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,-16.080927,1.490433,7.290475,12.770805,-2.401559,-0.143236,-9.110707,-0.044362,-22.508253,0.0000,-1.905429,17.536177,0.991342,-21.828601,-16.018098,-3.022159,0.229830,4.647811,-5.286930,-3.019935,-0.286837,1.681518,-31.103796,-8.774049,7.590720,-4.232454,8.458921,-7.666138,6.055294,-28.190132,15.473656,15.915722,-9.026852,1.171972,-5.391258,-59.082917,4.775361,1.834801,1.447475,2.394701,-4.988773,388.08,24.751632,9.235573,9.504289,-4.228346,-0.363838,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,8.955804,-2.612907,2.740679,9.704034,-1.524375,-4.001874,3.843455,0.358459,0.900835,-0.0001,-3.305960,11.099784,0.270078,-40.851392,-14.029412,-5.124856,-0.170633,-0.373501,2.284408,-2.855275,11.325377,3.110082,-19.147857,-8.226306,5.767890,6.093670,-7.830001,-6.567701,-3.929450,1.128238,3.463817,14.992986,-9.377136,-0.012398,4.087052,-61.945695,5.142323,12.855278,3.065369,-3.997100,6.548292,272.45,19.673348,-20.449157,-21.044141,-11.225027,9.176861,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,-1.243500,9.752493,-1.931690,1.224566,-18.678339,0.390264,10.615600,3.774666,-10.091425,-0.0001,1.023597,6.415119,-7.197756,-5.616595,10.993535,-2.497816,-0.196972,-12.111033,-0.197022,-1.789174,3.503165,5.784800,21.849473,-12.377823,0.727858,-5.169318,8.545741,1.213110,-9.659291,-12.638857,8.647821,10.183689,4.579249,0.586459,1.920605,13.387445,8.369600,-9.150596,-0.233386,4.280341,-3.315470,-127.28,-5.184808,4.179362,4.300964,-1.475613,1.628260,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39995,0.608853,-0.213463,-3.538605,10.957032,5.586005,-0.238700,16.942579,1.459900,-2.747632,-0.0001,-0.070824,-8.495781,0.955410,-1.825368,8.464736,0.988800,-0.121165,-7.987809,9.212888,0.855352,-0.310800,-3.211206,-6.657321,-3.572848,6.512648,6.255875,1.169474,7.945029,4.270322,-3.441232,11.050103,-2.555568,2.177949,-0.720993,6.142266,-7.251095,-8.217025,-5.909516,2.238280,7.453938,0.426583,1856.05,5.799848,6.348686,6.533406,-0.826225,-2.246642,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
39996,-15.654468,13.673730

In [13]:
### Orignially attempted different models with RandomForestRegressors and GridSearch methods
### Then, tested individual models to save on compute time resulting in the following model with best results

model_3_rfr = RandomForestClassifier(n_estimators=15000, 
                              criterion = "entropy",
                              n_jobs=-1, 
                              random_state=42, 
                              max_features=0.4, 
                              min_samples_leaf=1)
model_3_rfr.fit(X_train, y_train)
rocb = roc_auc_score(y_train, model_3_rfr.predict_proba(X_train)[:,1])
print ("C-stat: ", rocb)

df_model_3_rfr = pd.DataFrame(model_3_rfr.predict_proba(X_test)[:,1])
df_model_3_rfr.to_csv("y_hat_model_3_rfc15k.csv", index=True)

C-stat:  1.0
